In [80]:
import json
from scipy.stats import linregress
from statsmodels.stats.multitest import multipletests
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd

In [139]:
def read_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

            
def get_kegg_group(annotation, gene_list, target_group, level=0, 
                   group_found = False, level_found=None):
    """get a list of genes from kegg annotation
    """
    try:
        for group in annotation["children"]:
            # exit loop when back at the same level as the found group
            if level == level_found:
                group_found = False
                break

            name = group["name"]
            if name == target_group:
                group_found = True
                level_found = level

            if "; " in name and group_found:
                gene_list.append(name.split(" ")[0]  )
            elif "; " in name: # no need to go to the next level
                break
            else:
                get_kegg_group(group, gene_list, target_group, level=level+1, 
                               group_found=group_found, level_found=level_found)
                    
    except KeyError:
        pass
    
    

def get_kegg_group_names(annotation, names, wanted, level=0, parent=""):
    """get a list of groups from kegg annotation
    """
    # not_wanted = ['09130 Environmental Information Processing',
    #               '09140 Cellular Processes',
    #               '09150 Organismal Systems',
    #               '09160 Human Diseases',
    #               #'09180 Brite Hierarchies',
    #               '09190 Not Included in Pathway or Brite',
    #               '09185 Viral protein families',
    #               '09184 RNA family',
    #               '09125 Information processing in viruses']
    try:
        for group in annotation["children"]:
            name = group["name"]

           # groups that are not interesting
            # if name in not_wanted:
            #     continue

            if parent == "09100 Metabolism" and name not in wanted:
                name = "Metabolism Other"
            elif parent == "09120 Genetic Information Processing" and name not in wanted:
                name = "Genetic Information Processing Other"
            elif name not in wanted:
                name = "Other"

            if name.startswith("b"): # no need to go to the next level
                break
            else:
                try:
                    names[level].append(name)
                except KeyError:
                    names[level] = [name]
                get_kegg_group_names(group, names, wanted, level=level+1, parent=name)
                    
    except KeyError:
        pass

In [140]:
annotation = read_json_file("../data/eco00001.json")

In [141]:
interesting_groups = ['09100 Metabolism',
                  '09120 Genetic Information Processing',
                  '09180 Brite Hierarchies',
                    '09101 Carbohydrate metabolism',
                  '09102 Energy metabolism',
                  '09103 Lipid metabolism',
                  '09104 Nucleotide metabolism',
                  '09105 Amino acid metabolism',
                  '09106 Metabolism of other amino acids',
                     '09121 Transcription',
                     '09122 Translation',
                     '09123 Folding, sorting and degradation',
                      '09124 Replication and repair',
                     '09126 Chromosome',
                     '03010 Ribosome',
                     '00970 Aminoacyl-tRNA biosynthesis',
                     '03018 RNA degradation',
                     '09130 Environmental Information Processing',
                     '09131 Membrane transport',
                      '09182 Protein families: genetic information processing',
                     '03000 Transcription factors [BR:eco03000]',
                     '03021 Transcription machinery [BR:eco03021]',
                     '03019 Messenger RNA biogenesis',
                     '03011 Ribosome [BR:eco03011]',
                     '03009 Ribosome biogenesis [BR:eco03009]',
                     '03016 Transfer RNA biogenesis [BR:eco03016]',
                    '03012 Translation factors [BR:eco03012]',
                    '03110 Chaperones and folding catalysts [BR:eco03110]']

In [142]:
names={}
get_kegg_group_names(annotation, names, interesting_groups)

In [143]:
other_metabolism = ['09107 Glycan biosynthesis and metabolism',
  '09108 Metabolism of cofactors and vitamins',
  '09109 Metabolism of terpenoids and polyketides',
  '09110 Biosynthesis of other secondary metabolites',
  '09111 Xenobiotics biodegradation and metabolism',
 '09181 Protein families: metabolism',
   '00040 Pentose and glucuronate interconversions [PATH:eco00040]',
  '00051 Fructose and mannose metabolism [PATH:eco00051]',
  '00052 Galactose metabolism [PATH:eco00052]',
  '00053 Ascorbate and aldarate metabolism [PATH:eco00053]',
  '00500 Starch and sucrose metabolism [PATH:eco00500]',
  '00520 Amino sugar and nucleotide sugar metabolism [PATH:eco00520]',
  '00630 Glyoxylate and dicarboxylate metabolism [PATH:eco00630]',
  '00640 Propanoate metabolism [PATH:eco00640]',
  '00650 Butanoate metabolism [PATH:eco00650]',
  '00660 C5-Branched dibasic acid metabolism [PATH:eco00660]',
  '00562 Inositol phosphate metabolism [PATH:eco00562]',
  '00195 Photosynthesis',
  '00196 Photosynthesis - antenna proteins',
  '00710 Carbon fixation by Calvin cycle [PATH:eco00710]',
  '00720 Other carbon fixation pathways [PATH:eco00720]',
  '00680 Methane metabolism [PATH:eco00680]',
  '00910 Nitrogen metabolism [PATH:eco00910]',
  '00920 Sulfur metabolism [PATH:eco00920]',
  '00061 Fatty acid biosynthesis [PATH:eco00061]',
  '00062 Fatty acid elongation',
  '00071 Fatty acid degradation [PATH:eco00071]',
  '00073 Cutin, suberine and wax biosynthesis',
  '00074 Mycolic acid biosynthesis [PATH:eco00074]',
  '00100 Steroid biosynthesis',
  '00120 Primary bile acid biosynthesis',
  '00121 Secondary bile acid biosynthesis [PATH:eco00121]',
  '00140 Steroid hormone biosynthesis',
  '00561 Glycerolipid metabolism [PATH:eco00561]',
  '00564 Glycerophospholipid metabolism [PATH:eco00564]',
  '00565 Ether lipid metabolism [PATH:eco00565]',
  '00600 Sphingolipid metabolism [PATH:eco00600]',
  '00590 Arachidonic acid metabolism',
  '00591 Linoleic acid metabolism',
  '00592 alpha-Linolenic acid metabolism [PATH:eco00592]',
  '01040 Biosynthesis of unsaturated fatty acids [PATH:eco01040]',
  '00230 Purine metabolism [PATH:eco00230]',
  '00240 Pyrimidine metabolism [PATH:eco00240]',
  '00250 Alanine, aspartate and glutamate metabolism [PATH:eco00250]',
  '00260 Glycine, serine and threonine metabolism [PATH:eco00260]',
  '00270 Cysteine and methionine metabolism [PATH:eco00270]',
  '00280 Valine, leucine and isoleucine degradation [PATH:eco00280]',
  '00290 Valine, leucine and isoleucine biosynthesis [PATH:eco00290]',
  '00300 Lysine biosynthesis [PATH:eco00300]',
  '00310 Lysine degradation [PATH:eco00310]',
  '00220 Arginine biosynthesis [PATH:eco00220]',
  '00330 Arginine and proline metabolism [PATH:eco00330]',
  '00340 Histidine metabolism [PATH:eco00340]',
  '00350 Tyrosine metabolism [PATH:eco00350]',
  '00360 Phenylalanine metabolism [PATH:eco00360]',
  '00380 Tryptophan metabolism [PATH:eco00380]',
  '00400 Phenylalanine, tyrosine and tryptophan biosynthesis [PATH:eco00400]',
  '00410 beta-Alanine metabolism [PATH:eco00410]',
  '00430 Taurine and hypotaurine metabolism [PATH:eco00430]',
  '00440 Phosphonate and phosphinate metabolism [PATH:eco00440]',
  '00450 Selenocompound metabolism [PATH:eco00450]',
  '00460 Cyanoamino acid metabolism [PATH:eco00460]',
  '00470 D-Amino acid metabolism [PATH:eco00470]',
  '00480 Glutathione metabolism [PATH:eco00480]',
  '00510 N-Glycan biosynthesis',
  '00513 Various types of N-glycan biosynthesis',
  '00512 Mucin type O-glycan biosynthesis',
  '00515 Mannose type O-glycan biosynthesis',
  '00514 Other types of O-glycan biosynthesis',
  '00532 Glycosaminoglycan biosynthesis - chondroitin sulfate / dermatan sulfate',
  '00534 Glycosaminoglycan biosynthesis - heparan sulfate / heparin',
  '00533 Glycosaminoglycan biosynthesis - keratan sulfate',
  '00531 Glycosaminoglycan degradation',
  '00563 Glycosylphosphatidylinositol (GPI)-anchor biosynthesis',
  '00601 Glycosphingolipid biosynthesis - lacto and neolacto series',
  '00603 Glycosphingolipid biosynthesis - globo and isoglobo series',
  '00604 Glycosphingolipid biosynthesis - ganglio series',
  '00511 Other glycan degradation [PATH:eco00511]',
  '00540 Lipopolysaccharide biosynthesis [PATH:eco00540]',
  '00542 O-Antigen repeat unit biosynthesis [PATH:eco00542]',
  '00541 O-Antigen nucleotide sugar biosynthesis [PATH:eco00541]',
  '00550 Peptidoglycan biosynthesis [PATH:eco00550]',
  '00552 Teichoic acid biosynthesis [PATH:eco00552]',
  '00571 Lipoarabinomannan (LAM) biosynthesis',
  '00572 Arabinogalactan biosynthesis - Mycobacterium',
  '00543 Exopolysaccharide biosynthesis [PATH:eco00543]',
  '00730 Thiamine metabolism [PATH:eco00730]',
  '00740 Riboflavin metabolism [PATH:eco00740]',
  '00750 Vitamin B6 metabolism [PATH:eco00750]',
  '00760 Nicotinate and nicotinamide metabolism [PATH:eco00760]',
  '00770 Pantothenate and CoA biosynthesis [PATH:eco00770]',
  '00780 Biotin metabolism [PATH:eco00780]',
  '00785 Lipoic acid metabolism [PATH:eco00785]',
  '00790 Folate biosynthesis [PATH:eco00790]',
  '00670 One carbon pool by folate [PATH:eco00670]',
  '00830 Retinol metabolism',
  '00860 Porphyrin metabolism [PATH:eco00860]',
  '00130 Ubiquinone and other terpenoid-quinone biosynthesis [PATH:eco00130]',
  '00900 Terpenoid backbone biosynthesis [PATH:eco00900]',
  '00902 Monoterpenoid biosynthesis',
  '00909 Sesquiterpenoid and triterpenoid biosynthesis',
  '00904 Diterpenoid biosynthesis',
  '00906 Carotenoid biosynthesis',
  '00905 Brassinosteroid biosynthesis',
  '00981 Insect hormone biosynthesis',
  '00908 Zeatin biosynthesis',
  '00903 Limonene degradation',
  '00907 Pinene, camphor and geraniol degradation [PATH:eco00907]',
  '01052 Type I polyketide structures',
  '00522 Biosynthesis of 12-, 14- and 16-membered macrolides',
  '01051 Biosynthesis of ansamycins',
  '01059 Biosynthesis of enediyne antibiotics',
  '01056 Biosynthesis of type II polyketide backbone',
  '01057 Biosynthesis of type II polyketide products',
  '00253 Tetracycline biosynthesis',
  '00523 Polyketide sugar unit biosynthesis [PATH:eco00523]',
  '01054 Nonribosomal peptide structures',
  '01053 Biosynthesis of siderophore group nonribosomal peptides [PATH:eco01053]',
  '01055 Biosynthesis of vancomycin group antibiotics',
  '00940 Phenylpropanoid biosynthesis',
  '00945 Stilbenoid, diarylheptanoid and gingerol biosynthesis',
  '00941 Flavonoid biosynthesis',
  '00944 Flavone and flavonol biosynthesis',
  '00942 Anthocyanin biosynthesis',
  '00943 Isoflavonoid biosynthesis',
  '00946 Degradation of flavonoids [PATH:eco00946]',
  '00901 Indole alkaloid biosynthesis',
  '00403 Indole diterpene alkaloid biosynthesis',
  '00950 Isoquinoline alkaloid biosynthesis',
  '00960 Tropane, piperidine and pyridine alkaloid biosynthesis',
  '00996 Biosynthesis of various alkaloids',
  '00232 Caffeine metabolism',
  '00965 Betalain biosynthesis',
  '00966 Glucosinolate biosynthesis',
  '00402 Benzoxazinoid biosynthesis',
  '00311 Penicillin and cephalosporin biosynthesis',
  '00332 Carbapenem biosynthesis [PATH:eco00332]',
  '00261 Monobactam biosynthesis [PATH:eco00261]',
  '00331 Clavulanic acid biosynthesis',
  '00521 Streptomycin biosynthesis [PATH:eco00521]',
  '00524 Neomycin, kanamycin and gentamicin biosynthesis',
  '00525 Acarbose and validamycin biosynthesis [PATH:eco00525]',
  '00401 Novobiocin biosynthesis [PATH:eco00401]',
  '00404 Staurosporine biosynthesis',
  '00405 Phenazine biosynthesis',
  '00333 Prodigiosin biosynthesis',
  '00254 Aflatoxin biosynthesis',
  '00998 Biosynthesis of various antibiotics',
  '00999 Biosynthesis of various plant secondary metabolites [PATH:eco00999]',
  '00997 Biosynthesis of various other secondary metabolites',
  '00362 Benzoate degradation [PATH:eco00362]',
  '00627 Aminobenzoate degradation [PATH:eco00627]',
  '00364 Fluorobenzoate degradation [PATH:eco00364]',
  '00625 Chloroalkane and chloroalkene degradation [PATH:eco00625]',
  '00361 Chlorocyclohexane and chlorobenzene degradation [PATH:eco00361]',
  '00623 Toluene degradation [PATH:eco00623]',
  '00622 Xylene degradation [PATH:eco00622]',
  '00633 Nitrotoluene degradation [PATH:eco00633]',
  '00642 Ethylbenzene degradation',
  '00643 Styrene degradation',
  '00791 Atrazine degradation',
  '00930 Caprolactam degradation [PATH:eco00930]',
  '00363 Bisphenol degradation',
  '00621 Dioxin degradation [PATH:eco00621]',
  '00626 Naphthalene degradation [PATH:eco00626]',
  '00624 Polycyclic aromatic hydrocarbon degradation',
  '00365 Furfural degradation',
  '00984 Steroid degradation',
  '00980 Metabolism of xenobiotics by cytochrome P450',
  '00982 Drug metabolism - cytochrome P450',
  '00983 Drug metabolism - other enzymes']


In [144]:
not_wanted = ['09180 Brite Hierarchies']
other = ['09183 Protein families: signaling and cellular processes']

In [145]:
all_groups = {0: {},
              1: {},
              2: {}}
for level in all_groups:
    # assign genes to groups as keys
    temp_groups = {}
    for group in names[level]:
        gene_list=[]
        get_kegg_group(annotation, gene_list, group)
        temp_groups[group] = gene_list

    # flip it and assign groups to genes as keys
    genes_annot = {}
    for group, genes in temp_groups.items():
        group_name = group
        if group in other_metabolism:
            group_name = "Metabolism other"
        if group in not_wanted:
            continue
        if group in other:
            group_name = "Other"

        for gid in genes:
            try:
                genes_annot[gid].append(group_name)
            except KeyError:
                genes_annot[gid] = [group_name]

    # make a unique list of groups for each gene and count them
    n_groups = {}
    for gid, groups in genes_annot.items():
        all_groups[level][gid] = set(groups)
        try:
            n_groups[len(set(groups))] += 1
        except KeyError:
            n_groups[len(set(groups))] = 1

    print(f"Level {level} groups per gene: {n_groups}")


Level 0 groups per gene: {1: 1542, 2: 113}
Level 1 groups per gene: {1: 1374, 4: 10, 2: 390, 3: 33}
Level 2 groups per gene: {1: 520, 3: 1, 2: 6}


In [146]:
pd.DataFrame(all_groups).to_csv("../data/ecoli_kegg_annotation.csv")

In [148]:
all_groups[2]


{'b1961': {'03000 Transcription factors [BR:eco03000]'},
 'b0064': {'03000 Transcription factors [BR:eco03000]'},
 'b3906': {'03000 Transcription factors [BR:eco03000]'},
 'b3905': {'03000 Transcription factors [BR:eco03000]'},
 'b1735': {'03000 Transcription factors [BR:eco03000]'},
 'b4116': {'03000 Transcription factors [BR:eco03000]'},
 'b2437': {'03000 Transcription factors [BR:eco03000]'},
 'b0564': {'03000 Transcription factors [BR:eco03000]'},
 'b0566': {'03000 Transcription factors [BR:eco03000]'},
 'b4062': {'03000 Transcription factors [BR:eco03000]'},
 'b1531': {'03000 Transcription factors [BR:eco03000]'},
 'b4396': {'03000 Transcription factors [BR:eco03000]'},
 'b1384': {'03000 Transcription factors [BR:eco03000]'},
 'b0305': {'03000 Transcription factors [BR:eco03000]'},
 'b3515': {'03000 Transcription factors [BR:eco03000]'},
 'b3516': {'03000 Transcription factors [BR:eco03000]'},
 'b4118': {'03000 Transcription factors [BR:eco03000]'},
 'b1790': {'03000 Transcription